legge i file random attemps

In [ ]:
##### UPLOAD THE DATAFRAME FROM FILE
df = pd.read_csv(FILENAME, dtype = str).set_index('states')
df = df.astype(float).fillna(0.0)


##### COMPUTE PERCENTAGES AND ADD AVERAGE COLUMN
total = df.sum()
percentage = (df/ total) * 100
# print(percentage.iloc[:,0].sum()) # check

# percentage["states"] = percentage.index
percentage['average'] = percentage.mean(numeric_only=True, axis=1)
percentage['std'] = percentage[percentage.columns[:-1]].std(numeric_only=True, axis=1)

##### FIND THE BEST HISTOGRAM INDEX (i_best) BASED ON THE MINIMUM ENERGY FOUND
with open(DATA_FILENAME, 'r') as DATA_FILE:
    for line in DATA_FILE:
        if 'Attempt that reached the best result with' in line:                
            string = line.split('#')[1]
            i_best = string.split(' ')[0]
            print(i_best)
            
##### THE COLUMN CORRESPONDING TO THE BEST RESULT
column_best = f'counts_p{p}_{i_best}of{random_attempts}'


##### KEEP THE BEST AND THE AVERAGE RESULT
percentage = percentage[[column_best, "average", "std"]]
percentage = percentage.sort_values(column_best, ascending=False)
df1 = percentage[["average", "std"]].rename(columns={"average": "y"})
df2 =  percentage[[column_best]].rename(columns={column_best: "y"})

df1["hue"] = f"Average on {random_attempts} random attempts"
df2["hue"] = f"Lowest energy among {random_attempts} random attempts"

df_barplot = pd.concat([df1,df2])

##### FIGURE
N=11
plt.figure(figsize=(15,5))
ax = sns.barplot(x="states", y="y", hue="hue", data=df_barplot, width=0.9, palette="deep")

# Make labels with percentages.
labels_avg = percentage["average"].round(1).astype('str') + '%'
labels_best = percentage[column_best].round(1).astype('str') + '%'
for i,container in enumerate(ax.containers):
    if i%2 == 0:
        ax.bar_label(container, labels_avg, fontsize=N-2)
    if i%2 == 1:
        ax.bar_label(container, labels_best, fontsize=N-2)
        
# Display errors, but just for the "average" bars.
num_of_avg_bars = len(percentage) # number of "average" bars
x_coords = [p.get_x() + 0.5 * p.get_width() for p in ax.patches[:num_of_avg_bars]] 
y_coords = [p.get_height() for p in ax.patches[:num_of_avg_bars]]
ax.errorbar(x=x_coords, y=y_coords, yerr=percentage["std"], fmt="none", c="k", capsize=3.5, alpha=0.4)

# Highlight exact covers' ticks with red.
df_for_ticks = percentage.copy()
df_for_ticks["states"] = df_for_ticks.index
highlight_correct_ticks(ax, df_for_ticks, U, subsets_dict)

# Plot refinements.
plt.title(FILENAME, fontsize=N+4)
plt.xlabel("states", fontsize=N)
plt.ylabel("", fontsize=N)
plt.xticks(fontsize=N)
plt.yticks(fontsize=N)
plt.xlim(xmin=-1)
plt.ylim(ymin=0)
plt.minorticks_on()
plt.grid(alpha=0.2)
plt.legend(fontsize=N)
plt.show()


Grafici iniziali

In [ ]:
# Find the exact cover 
mask = df_final['states'].apply(lambda x : compute_energy_Lucas(x, U, subsets_dict)) == 0
print(df_final.head())
print(mask)

df_exact_covers = df_final[mask]
print(df_exact_covers.head())
exact_covers = df_exact_covers["states"].values
print(exact_covers)
solution =  df_exact_covers["states"].iloc[0]
print(solution)
counts = df_exact_covers.iloc[0, 1:].values
layers = [str(i+1) for i in np.arange(max_p)]

# Plot the figure.
plt.figure()
total = 1024
plt.plot(layers, counts*100/total, linestyle='--', color='k', marker='o', label=f"{solution}")
plt.ylabel("Occurrences [%]")
plt.xlabel("p (number of layers)")
plt.legend()

plt.show()

### usato per stimare k

In [ ]:
states_that_respect_constraint = []

for nuple in bit_gen(PROBLEM_DIM):
    
    state = "".join([str(bit) for bit in nuple]) # strings
    chosen_subsets = [subsets[i] for i,digit in enumerate(state) if digit=="1"]
    u = set().union(*chosen_subsets)
    sum_of_len = sum([len(sub) for sub in chosen_subsets])

    """La somma delle lunghezze dei sottoinsiemi selezionati da uno stato
       è uguale alla lunghezza dell'insieme unione u solo 
       se i sottoinsiemi non si intersecano, 
       ovvero se lo stato soddisfa il vincolo.
    """
    if len(u) == sum_of_len:        
        states_that_respect_constraint.append(state)


#### ASSOCIO A OGNI STATO IL PESO 
vec=[]
for s in states_that_respect_constraint:
    tot=0
    for pos,digit in enumerate(s):
        if digit =="1":
            tot += len(subsets[pos])
    print(tot)
    vec.append(tot)

d = {(state, w) for state,w in zip(states_that_respect_constraint, vec)}



#### TROVO STATI CON DUE SOLI "1"
for elem in d:
    if elem[0].count("1") == 2:
        print(elem)


##################################
maxA = max([x.count("1") for x in states_that_respect_constraint])

t = max(lengths)

print("L=", maxA,"t=", t)
print((maxA - 1)/(len(U)-t)) 
k_try = ((maxA - 1)/(len(U)-t)) / PROBLEM_DIM

print(f"k_try = {k_try}  =>  l1/l2 > {k_try} n")

def my_ceil(a, precision=0):
    return np.true_divide(np.ceil(a * 10**precision), 10**precision)

def my_floor(a, precision=0):
    return np.true_divide(np.floor(a * 10**precision), 10**precision)

k = my_ceil(k_try,3)
if (k_try == k):
    k = k_try + 0.01

print(f"k={k}, k_try={k_try}")